In [1]:
!export LC_ALL=en_US.UTF-8
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')
np.random.seed(0)

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = True

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,left, right,cap_length = 20):
    y = np.zeros(cap_length)
    left = max(0,left - int((cap_length - len(answer))/2))
    right = min(right + int((cap_length + len(answer))/2), cap_length)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
n_words = 10
X_train_comp_all = []
X_train_comp_with_answer_marked_all = []
X_train_ans_all = []
X_train_comp_answer_label_all = []
X_train_sentence_label_all = []
Y_train_ques_all = []
invalid = 0
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())

    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                invalid = invalid+1
                continue
            marked_comp = np.zeros(len(passage))
            marked_comp[start:end] = 1
            left = max(0,start - n_words)
            right = min(len(passage), end + n_words)
            
            cappedPassage = passage[left:right]
            marked_comp = marked_comp[left:right]
            
            
            X_train_comp_all.append(cappedPassage)
            X_train_comp_with_answer_marked_all.append(marked_comp)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
print(invalid)

2429


In [8]:
c = list(zip(X_train_comp_all,X_train_ans_all, Y_train_ques_all,X_train_comp_with_answer_marked_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_ans_all_shuffled, Y_train_ques_all_shuffled, X_train_comp_with_answer_marked_all_shuffled = zip(*c)


In [9]:
X_train_comp_with_answer_marked_all_shuffled[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [174]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all_shuffled for item in sublist] + [item for sublist in Y_train_ques_all_shuffled for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [205]:
wordToTake = 200000
words = findKMostFrequentWords(100000)

In [206]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(59528, 300)

In [207]:
!hostname
print(invalid)

cuda2.cims.nyu.edu
2429


In [208]:
examples_to_take_train = 32000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_all_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
X_train_comp_with_answer_marked = X_train_comp_with_answer_marked_all_shuffled[0:examples_to_take_train]

In [209]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [210]:
document_tokens = np.full((examples_to_take_train, max_document_len), END_TOKEN,dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)


In [211]:
for i in range(examples_to_take_train):
    answer_labels[i, 0:len(X_train_comp_with_answer_marked[i])] = X_train_comp_with_answer_marked[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    answer_lengths[i] = len(X_train_ans[i])
    
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)    

In [212]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [213]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(59528, 300)
51426
4644
49775


In [214]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [215]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [216]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [217]:
max_document_len

66

## Batch Data Preparation

In [218]:
max_question_len

40

In [219]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[]}
        
        for index,inp in enumerate(inputs):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[3][start:start+batch_size])
            maxQ = max_question_len
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_lengths'].append(inp[start:end])
            elif index==4:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==5:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_lengths'].append(inp[start:end])
        
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_lengths"] = np.array(output["answer_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [220]:
batch_size = 128
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)
for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze()
number_of_batches = len(batch_input)
print("Number of batches = ", number_of_batches)

Number of batches =  250


In [221]:
split = int(0.8 * number_of_batches)
batch_input_test = batch_input[split:]
batch_input = batch_input[0:split]

In [222]:
len(batch_input)

200

## Model

In [223]:
len(batch_input)

200

In [224]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import time

In [225]:
use_cuda = torch.cuda.is_available()
use_attention = True

In [226]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [227]:
class DocumentEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(DocumentEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self, batch_size):
        result = Variable(torch.zeros(2, batch_size, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [228]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        if use_cuda:
            self.fc1 = self.fc1.cuda()
            self.fc2 = self.fc2.cuda()

    def forward(self, x):
        output = self.fc1(x)
        output = F.tanh(output)
        output = self.fc2(output)
        output = F.log_softmax(output, dim=-1)
        return output

In [229]:
class Attention(nn.Module):
    def __init__(self, input_size, output_size):
        super(Attention,self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, hidden, encoder_outputs):
        #hidden = self.dropout(hidden)

        encoder_outputs = self.fc(encoder_outputs)
        
        activations = torch.bmm(encoder_outputs, hidden.unsqueeze(1).transpose(1,2))
        activations = F.softmax(activations, dim=1)
        output = torch.mul(encoder_outputs, activations)
        output = torch.sum(output, dim=1)
        output = F.relu(output)
        return output.unsqueeze(0)

In [230]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
documentEncoder = DocumentEncoderRNN(input_size = hidden_size+1, hidden_size = hidden_size)
questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=2 * hidden_size)
questionGenerator = QuestionGenerationFC(input_size = 2*questionDecoder.hidden_size, hidden_size=2*hidden_size, output_size=reduced_glove.shape[0])
attention = Attention(2 * documentEncoder.hidden_size, questionDecoder.hidden_size)

documentEncoder.train()
questionDecoder.train()
questionGenerator.train()
attention.train()

train_param = []

for model in [documentEncoder, questionDecoder, questionGenerator, attention]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 4e-3)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

Number of trainable parameters =  20


In [231]:
def trainEpoch(verboseBatchPrinting):
    #verboseBatchPrinting = True
    avg_loss = 0
    epoch_time_start = time.time()
    for batch_num in range(len(batch_input)):
        batch_time_start = time.time()
        document_encoder_hidden = documentEncoder.initHidden(batch_size)

        current_batch_size = len(batch_input[batch_num]["document_tokens"])
        if current_batch_size != batch_size:
            continue

        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))

        for i in range(current_batch_size):
            mask[i,0:batch_input[batch_num]["document_lengths"][i]] = 1


        inp = Variable(torch.from_numpy(batch_input[batch_num]["document_tokens"]).long())


        optimizer.zero_grad()
        embedded_inp = embedder(inp)

        if use_cuda:
            embedded_inp = embedded_inp.cuda()
            
        #Adding answer=0/1 dim to embedded inp  
        answer_target = Variable(torch.from_numpy(batch_input[batch_num]['answer_labels']).float())
        if use_cuda:
            answer_target = answer_target.cuda()
        embedded_inp = torch.cat((embedded_inp,answer_target.unsqueeze(-1)),dim=-1)

        answer_tags, answer_outputs, document_encoder_hidden = documentEncoder(embedded_inp, document_encoder_hidden)              


        question_decoder_hidden = document_encoder_hidden.view(1,document_encoder_hidden.shape[1],2*document_encoder_hidden.shape[2])
        
        embedded_inputs = embedder(torch.from_numpy(batch_input[batch_num]["question_input_tokens"]).long())
        output_labels = Variable(torch.from_numpy(batch_input[batch_num]["question_output_tokens"]).long())
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()

        question_loss = 0
        for quesL in range(batch_input[batch_num]["question_input_tokens"].shape[1]):
            _, question_decoder_hidden = questionDecoder(
                embedded_inputs[:,quesL:quesL+1,:],
                question_decoder_hidden)


            output = question_decoder_hidden
            if use_attention:
                attn_output = attention(question_decoder_hidden.squeeze(0), answer_outputs)
                output = torch.cat((attn_output, output), dim=2)


            final_output = questionGenerator(output.squeeze(0))
            question_word_loss = criterion2(final_output,
                                        output_labels[:, quesL:quesL+1].squeeze(1))
            question_loss += question_word_loss

        net_loss = question_loss
        net_loss.backward()
        optimizer.step()


        avg_loss+= net_loss.data[0]
        batch_time_end = time.time()

        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tQuestion Loss: %.4f \t Time Taken: %d seconds'
                   %(batch_num, epoch, net_loss.data[0],  question_loss.data[0], batch_time_end-batch_time_start))
    epoch_time_end = time.time()
    
    return avg_loss/len(batch_input), epoch_time_end - epoch_time_start

In [232]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100


for epoch in range(1, num_epochs+1):
    
    avg_loss, time_taken = trainEpoch(verboseBatchPrinting)
    

    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f Time Taken: %d seconds'
                   %(epoch, avg_loss, time_taken))
        

Batch: 0 	 Epoch : 1	Net Loss: 440.5264 	Question Loss: 440.5264 	 Time Taken: 0 seconds
Batch: 1 	 Epoch : 1	Net Loss: 279.1686 	Question Loss: 279.1686 	 Time Taken: 0 seconds
Batch: 2 	 Epoch : 1	Net Loss: 125.3199 	Question Loss: 125.3199 	 Time Taken: 0 seconds
Batch: 3 	 Epoch : 1	Net Loss: 141.9577 	Question Loss: 141.9577 	 Time Taken: 0 seconds
Batch: 4 	 Epoch : 1	Net Loss: 89.8704 	Question Loss: 89.8704 	 Time Taken: 0 seconds
Batch: 5 	 Epoch : 1	Net Loss: 89.7388 	Question Loss: 89.7388 	 Time Taken: 0 seconds
Batch: 6 	 Epoch : 1	Net Loss: 90.0757 	Question Loss: 90.0757 	 Time Taken: 0 seconds
Batch: 7 	 Epoch : 1	Net Loss: 93.7322 	Question Loss: 93.7322 	 Time Taken: 0 seconds
Batch: 8 	 Epoch : 1	Net Loss: 94.9184 	Question Loss: 94.9184 	 Time Taken: 0 seconds
Batch: 9 	 Epoch : 1	Net Loss: 96.0194 	Question Loss: 96.0194 	 Time Taken: 0 seconds
Batch: 10 	 Epoch : 1	Net Loss: 90.7206 	Question Loss: 90.7206 	 Time Taken: 0 seconds
Batch: 11 	 Epoch : 1	Net Loss: 94

Batch: 94 	 Epoch : 1	Net Loss: 90.4320 	Question Loss: 90.4320 	 Time Taken: 0 seconds
Batch: 95 	 Epoch : 1	Net Loss: 90.3221 	Question Loss: 90.3221 	 Time Taken: 0 seconds
Batch: 96 	 Epoch : 1	Net Loss: 88.7474 	Question Loss: 88.7474 	 Time Taken: 0 seconds
Batch: 97 	 Epoch : 1	Net Loss: 93.0445 	Question Loss: 93.0445 	 Time Taken: 0 seconds
Batch: 98 	 Epoch : 1	Net Loss: 89.3115 	Question Loss: 89.3115 	 Time Taken: 0 seconds
Batch: 99 	 Epoch : 1	Net Loss: 89.9095 	Question Loss: 89.9095 	 Time Taken: 0 seconds
Batch: 100 	 Epoch : 1	Net Loss: 89.7517 	Question Loss: 89.7517 	 Time Taken: 0 seconds
Batch: 101 	 Epoch : 1	Net Loss: 92.4366 	Question Loss: 92.4366 	 Time Taken: 0 seconds
Batch: 102 	 Epoch : 1	Net Loss: 89.8584 	Question Loss: 89.8584 	 Time Taken: 0 seconds
Batch: 103 	 Epoch : 1	Net Loss: 89.8544 	Question Loss: 89.8544 	 Time Taken: 0 seconds
Batch: 104 	 Epoch : 1	Net Loss: 92.7573 	Question Loss: 92.7573 	 Time Taken: 0 seconds
Batch: 105 	 Epoch : 1	Net 

Batch: 187 	 Epoch : 1	Net Loss: 94.6967 	Question Loss: 94.6967 	 Time Taken: 0 seconds
Batch: 188 	 Epoch : 1	Net Loss: 89.6742 	Question Loss: 89.6742 	 Time Taken: 0 seconds
Batch: 189 	 Epoch : 1	Net Loss: 90.1229 	Question Loss: 90.1229 	 Time Taken: 0 seconds
Batch: 190 	 Epoch : 1	Net Loss: 91.3001 	Question Loss: 91.3001 	 Time Taken: 0 seconds
Batch: 191 	 Epoch : 1	Net Loss: 90.0968 	Question Loss: 90.0968 	 Time Taken: 0 seconds
Batch: 192 	 Epoch : 1	Net Loss: 95.0068 	Question Loss: 95.0068 	 Time Taken: 0 seconds
Batch: 193 	 Epoch : 1	Net Loss: 93.3493 	Question Loss: 93.3493 	 Time Taken: 0 seconds
Batch: 194 	 Epoch : 1	Net Loss: 95.1813 	Question Loss: 95.1813 	 Time Taken: 0 seconds
Batch: 195 	 Epoch : 1	Net Loss: 87.8377 	Question Loss: 87.8377 	 Time Taken: 0 seconds
Batch: 196 	 Epoch : 1	Net Loss: 88.9911 	Question Loss: 88.9911 	 Time Taken: 0 seconds
Batch: 197 	 Epoch : 1	Net Loss: 94.8126 	Question Loss: 94.8126 	 Time Taken: 0 seconds
Batch: 198 	 Epoch : 

Batch: 80 	 Epoch : 2	Net Loss: 93.3429 	Question Loss: 93.3429 	 Time Taken: 0 seconds
Batch: 81 	 Epoch : 2	Net Loss: 91.1479 	Question Loss: 91.1479 	 Time Taken: 0 seconds
Batch: 82 	 Epoch : 2	Net Loss: 88.9256 	Question Loss: 88.9256 	 Time Taken: 0 seconds
Batch: 83 	 Epoch : 2	Net Loss: 87.9771 	Question Loss: 87.9771 	 Time Taken: 0 seconds
Batch: 84 	 Epoch : 2	Net Loss: 91.4272 	Question Loss: 91.4272 	 Time Taken: 0 seconds
Batch: 85 	 Epoch : 2	Net Loss: 89.9016 	Question Loss: 89.9016 	 Time Taken: 0 seconds
Batch: 86 	 Epoch : 2	Net Loss: 91.8575 	Question Loss: 91.8575 	 Time Taken: 0 seconds
Batch: 87 	 Epoch : 2	Net Loss: 91.0882 	Question Loss: 91.0882 	 Time Taken: 0 seconds
Batch: 88 	 Epoch : 2	Net Loss: 93.8736 	Question Loss: 93.8736 	 Time Taken: 0 seconds
Batch: 89 	 Epoch : 2	Net Loss: 89.6873 	Question Loss: 89.6873 	 Time Taken: 0 seconds
Batch: 90 	 Epoch : 2	Net Loss: 94.6565 	Question Loss: 94.6565 	 Time Taken: 0 seconds
Batch: 91 	 Epoch : 2	Net Loss: 

Batch: 173 	 Epoch : 2	Net Loss: 90.8101 	Question Loss: 90.8101 	 Time Taken: 0 seconds
Batch: 174 	 Epoch : 2	Net Loss: 95.6715 	Question Loss: 95.6715 	 Time Taken: 0 seconds
Batch: 175 	 Epoch : 2	Net Loss: 98.9076 	Question Loss: 98.9076 	 Time Taken: 0 seconds
Batch: 176 	 Epoch : 2	Net Loss: 90.8460 	Question Loss: 90.8460 	 Time Taken: 0 seconds
Batch: 177 	 Epoch : 2	Net Loss: 92.5631 	Question Loss: 92.5631 	 Time Taken: 0 seconds
Batch: 178 	 Epoch : 2	Net Loss: 91.0652 	Question Loss: 91.0652 	 Time Taken: 0 seconds
Batch: 179 	 Epoch : 2	Net Loss: 92.4652 	Question Loss: 92.4652 	 Time Taken: 0 seconds
Batch: 180 	 Epoch : 2	Net Loss: 90.6703 	Question Loss: 90.6703 	 Time Taken: 0 seconds
Batch: 181 	 Epoch : 2	Net Loss: 93.3036 	Question Loss: 93.3036 	 Time Taken: 0 seconds
Batch: 182 	 Epoch : 2	Net Loss: 94.6529 	Question Loss: 94.6529 	 Time Taken: 0 seconds
Batch: 183 	 Epoch : 2	Net Loss: 90.2578 	Question Loss: 90.2578 	 Time Taken: 0 seconds
Batch: 184 	 Epoch : 

...

Batch: 46 	 Epoch : 50	Net Loss: 92.1037 	Question Loss: 92.1037 	 Time Taken: 0 seconds
Batch: 47 	 Epoch : 50	Net Loss: 97.1831 	Question Loss: 97.1831 	 Time Taken: 0 seconds
Batch: 48 	 Epoch : 50	Net Loss: 86.9021 	Question Loss: 86.9021 	 Time Taken: 0 seconds
Batch: 49 	 Epoch : 50	Net Loss: 94.4067 	Question Loss: 94.4067 	 Time Taken: 0 seconds
Batch: 50 	 Epoch : 50	Net Loss: 92.9496 	Question Loss: 92.9496 	 Time Taken: 0 seconds
Batch: 51 	 Epoch : 50	Net Loss: 88.5372 	Question Loss: 88.5372 	 Time Taken: 0 seconds
Batch: 52 	 Epoch : 50	Net Loss: 89.5697 	Question Loss: 89.5697 	 Time Taken: 0 seconds
Batch: 53 	 Epoch : 50	Net Loss: 90.0012 	Question Loss: 90.0012 	 Time Taken: 0 seconds
Batch: 54 	 Epoch : 50	Net Loss: 96.4641 	Question Loss: 96.4641 	 Time Taken: 0 seconds
Batch: 55 	 Epoch : 50	Net Loss: 93.1361 	Question Loss: 93.1361 	 Time Taken: 0 seconds
Batch: 56 	 Epoch : 50	Net Loss: 94.3989 	Question Loss: 94.3989 	 Time Taken: 0 seconds
Batch: 57 	 Epoch : 5

Batch: 138 	 Epoch : 50	Net Loss: 93.3195 	Question Loss: 93.3195 	 Time Taken: 0 seconds
Batch: 139 	 Epoch : 50	Net Loss: 92.0521 	Question Loss: 92.0521 	 Time Taken: 0 seconds
Batch: 140 	 Epoch : 50	Net Loss: 89.3162 	Question Loss: 89.3162 	 Time Taken: 0 seconds
Batch: 141 	 Epoch : 50	Net Loss: 89.7517 	Question Loss: 89.7517 	 Time Taken: 0 seconds
Batch: 142 	 Epoch : 50	Net Loss: 94.5487 	Question Loss: 94.5487 	 Time Taken: 0 seconds
Batch: 143 	 Epoch : 50	Net Loss: 89.2899 	Question Loss: 89.2899 	 Time Taken: 0 seconds
Batch: 144 	 Epoch : 50	Net Loss: 86.9231 	Question Loss: 86.9231 	 Time Taken: 0 seconds
Batch: 145 	 Epoch : 50	Net Loss: 95.3363 	Question Loss: 95.3363 	 Time Taken: 0 seconds
Batch: 146 	 Epoch : 50	Net Loss: 98.6982 	Question Loss: 98.6982 	 Time Taken: 0 seconds
Batch: 147 	 Epoch : 50	Net Loss: 86.9436 	Question Loss: 86.9436 	 Time Taken: 0 seconds
Batch: 148 	 Epoch : 50	Net Loss: 96.5871 	Question Loss: 96.5871 	 Time Taken: 0 seconds
Batch: 149

Batch: 29 	 Epoch : 51	Net Loss: 89.9133 	Question Loss: 89.9133 	 Time Taken: 0 seconds
Batch: 30 	 Epoch : 51	Net Loss: 91.6474 	Question Loss: 91.6474 	 Time Taken: 0 seconds
Batch: 31 	 Epoch : 51	Net Loss: 90.4685 	Question Loss: 90.4685 	 Time Taken: 0 seconds
Batch: 32 	 Epoch : 51	Net Loss: 88.7984 	Question Loss: 88.7984 	 Time Taken: 0 seconds
Batch: 33 	 Epoch : 51	Net Loss: 93.3062 	Question Loss: 93.3062 	 Time Taken: 0 seconds
Batch: 34 	 Epoch : 51	Net Loss: 87.3558 	Question Loss: 87.3558 	 Time Taken: 0 seconds
Batch: 35 	 Epoch : 51	Net Loss: 90.1345 	Question Loss: 90.1345 	 Time Taken: 0 seconds
Batch: 36 	 Epoch : 51	Net Loss: 93.7949 	Question Loss: 93.7949 	 Time Taken: 0 seconds
Batch: 37 	 Epoch : 51	Net Loss: 95.4328 	Question Loss: 95.4328 	 Time Taken: 0 seconds
Batch: 38 	 Epoch : 51	Net Loss: 91.2832 	Question Loss: 91.2832 	 Time Taken: 0 seconds
Batch: 39 	 Epoch : 51	Net Loss: 93.8742 	Question Loss: 93.8742 	 Time Taken: 0 seconds
Batch: 40 	 Epoch : 5

Batch: 121 	 Epoch : 51	Net Loss: 93.8885 	Question Loss: 93.8885 	 Time Taken: 0 seconds
Batch: 122 	 Epoch : 51	Net Loss: 96.8583 	Question Loss: 96.8583 	 Time Taken: 0 seconds
Batch: 123 	 Epoch : 51	Net Loss: 92.7146 	Question Loss: 92.7146 	 Time Taken: 0 seconds
Batch: 124 	 Epoch : 51	Net Loss: 94.6840 	Question Loss: 94.6840 	 Time Taken: 0 seconds
Batch: 125 	 Epoch : 51	Net Loss: 92.8001 	Question Loss: 92.8001 	 Time Taken: 0 seconds
Batch: 126 	 Epoch : 51	Net Loss: 96.4561 	Question Loss: 96.4561 	 Time Taken: 0 seconds
Batch: 127 	 Epoch : 51	Net Loss: 88.0429 	Question Loss: 88.0429 	 Time Taken: 0 seconds
Batch: 128 	 Epoch : 51	Net Loss: 93.2485 	Question Loss: 93.2485 	 Time Taken: 0 seconds
Batch: 129 	 Epoch : 51	Net Loss: 93.1616 	Question Loss: 93.1616 	 Time Taken: 0 seconds
Batch: 130 	 Epoch : 51	Net Loss: 100.1947 	Question Loss: 100.1947 	 Time Taken: 0 seconds
Batch: 131 	 Epoch : 51	Net Loss: 94.8028 	Question Loss: 94.8028 	 Time Taken: 0 seconds
Batch: 1

KeyboardInterrupt: 

In [ ]:
def save(path):
    d = dict()
    documentEncoder.train()
    questionDecoder.train()
    questionGenerator.train()
    attention.train()
    d['documentEncoder'] = documentEncoder.state_dict()
    d['questionDecoder'] = questionDecoder.state_dict()
    d['questionGenerator'] = questionGenerator.state_dict()
    d['attention'] = attention.state_dict()
    d['optimizer'] = optimizer.state_dict()
    torch.save(d, path)
save("/data/ra2630/32k_model.pt")

In [138]:
checkpoint = torch.load("/data/ra2630/3k_model.pt")

In [ ]:
reduced_glove.shape[1]

In [233]:
def prepareSetForInferenceFromBatch(batch_number):
    data_dict = batch_input[batch_number]
    data_dict['answer_mask_1'] = data_dict['answer_labels']
    data_dict['answer_mask_2'] = 1 - data_dict['answer_mask_1']
    return data_dict

In [234]:
def prepareSetForInference(context, answer, maxDocLength, batch_size):
    
    passage = word_tokenize(context.lower())
    doc_len = len(passage)
    
    ans_condition = word_tokenize(answer.lower())
    
    ans_len = len(ans_condition)
        
    start,end = find_sub_list(ans_condition,passage)

    if start == -1:
            print("Couldn't Find answer in text, Please try again !!")
            return None
    
    
    document_token = np.full((1, maxDocLength), END_TOKEN, dtype=np.int32)
    document_length = np.zeros(1, dtype=np.int32)
    answer_mask_1 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_mask_2 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_length = np.zeros(1, dtype=np.int32)
    
    document_length[0] = doc_len
    answer_length[0] = ans_len
    answer_mask_1[0,start:end] = 1
    answer_mask_2 = 1 - answer_mask_1
    
    
    for i in range(doc_len):
        document_token[:,i] = look_up_word_reduced(passage[i])
    
    document_token = document_token.repeat(batch_size, axis = 0)
    document_length = document_length.repeat(batch_size, axis = 0)
    answer_mask_1 = answer_mask_1.repeat(batch_size, axis = 0)
    answer_mask_2 = answer_mask_2.repeat(batch_size, axis = 0)
    answer_length = answer_length.repeat(batch_size, axis = 0)
    
    
    return {
        "document_tokens" : document_token,
        "document_length" : document_length,
        "answer_mask_1" : answer_mask_1,
        "answer_mask_2" : answer_mask_2,
        "answer_length" : answer_length,
    }

In [235]:
def inference(data_dict):
    
    hidden_size = reduced_glove.shape[1]
    embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
    documentEncoder2 = DocumentEncoderRNN(input_size = hidden_size+1, hidden_size = hidden_size)
    questionDecoder2 = QuestionDecoderRNN(input_size=hidden_size, hidden_size=2 * hidden_size)
    questionGenerator2 = QuestionGenerationFC(input_size = 2*questionDecoder2.hidden_size, hidden_size=2*hidden_size, output_size=reduced_glove.shape[0])
    attention2 = Attention(2 * documentEncoder2.hidden_size, questionDecoder2.hidden_size)
    optimizer2 = torch.optim.Adam(train_param)


    use_ckpt = False

    if use_ckpt:
        documentEncoder2.load_state_dict(checkpoint["documentEncoder"])
        questionDecoder2.load_state_dict(checkpoint["questionDecoder"])
        questionGenerator2.load_state_dict(checkpoint["questionGenerator"])
        attention2.load_state_dict(checkpoint["attention"])
        optimizer2.load_state_dict(checkpoint["optimizer"])
    else:
        documentEncoder2 = documentEncoder
        questionDecoder2 = questionDecoder
        questionGenerator2 = questionGenerator
        attention2 = attention
        optimizer2 = optimizer


    documentEncoder2.eval()
    questionDecoder2.eval()
    questionGenerator2.eval()
    attention2.eval()
    
    use_attention = True
    use_cuda = True

    document_encoder_hidden_inf = documentEncoder2.initHidden(batch_size)


    inp = Variable(torch.from_numpy(data_dict["document_tokens"]).long())
    labels = Variable(torch.from_numpy(data_dict["answer_mask_1"])).long() #Let's see if we want to use or not
    embedded_inp = embedder(inp)


    if use_cuda:
        labels = labels.cuda()
        embedded_inp = embedded_inp.cuda()
    #Adding answer=0/1 dim to embedded inp  
    embedded_inp = torch.cat((embedded_inp,labels.unsqueeze(-1).float()),dim=-1)
    answer_tags, answer_outputs, document_encoder_hidden_inf = documentEncoder2(embedded_inp, document_encoder_hidden_inf)

    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda()

      
    maxGenQuesLen = 20
    currGenQuestionLen = 0
    
    question_decoder_hidden_inf = document_encoder_hidden_inf.view(1,document_encoder_hidden_inf.shape[1],2*document_encoder_hidden_inf.shape[2])

    current_question_token = torch.from_numpy(np.full((batch_size), START_TOKEN)).long()
    questionGenerated = []

    for i in range(batch_size):
        questionGenerated.append([])
    
    
    while currGenQuestionLen < maxGenQuesLen:
        question_token_embedding = embedder(current_question_token)
        if use_cuda:
            question_token_embedding = question_token_embedding.cuda()
        
        _, question_decoder_hidden_inf = questionDecoder(
            question_token_embedding.unsqueeze(1),
            question_decoder_hidden_inf)


        output_inf = question_decoder_hidden_inf
        
        #if use_attention:
        attn_output_inf = attention(question_decoder_hidden_inf.squeeze(0), answer_outputs)
        output_inf = torch.cat((attn_output_inf, output_inf), dim=2)

        final_output = questionGenerator2(output_inf.squeeze(0))
        current_question_token = np.argmax(final_output.data, axis = 1)
        for i in range(batch_size):
            questionGenerated[i].append(look_up_token_reduced(current_question_token[i]))
        currGenQuestionLen += 1

    return answer_tags, questionGenerated

In [237]:
b_n = 1
context = ' '.join(X_train_comp_all_shuffled[10])
answer = ' '.join(X_train_ans_all_shuffled[10])

data_dict1 = prepareSetForInference(context, answer, max_document_len, batch_size)
data_dict2 = prepareSetForInferenceFromBatch(b_n)
    
a_tags1, questionGenerated1 = inference(data_dict1)
a_tags2, questionGenerated2 = inference(data_dict2)

for i in range(batch_size):
    print("------------------------------------------------------------------------------------------")
    print("Comprehension : ", ' '.join(X_train_comp_all_shuffled[32*b_n + i]))
    print("Ground Truth Question: ", ' '.join(Y_train_ques_all_shuffled[32*b_n + i]))
    print("Question Generated : ", questionGenerated2[i])
    print("Answer : ", ' '.join(X_train_ans_all_shuffled[32*b_n + i]))
    print("------------------------------------------------------------------------------------------")     


------------------------------------------------------------------------------------------
Comprehension :  , shortly after the end of world war ii , harry s. truman signed an executive order dissolving the oss , and by
Ground Truth Question:  who signed the order to get rid of the oss ?
Question Generated :  ['<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<END>']
Answer :  harry s. truman
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
Comprehension :  written music , transmission was oral , and subject to change every time it was transmitted . with a musical score
Ground Truth Question:  what was oral music subject to every time is was transmitted ?
Question Generated :  ['<END>', '<END>', '<END>', '<END>', '<END>', '<END>', '<EN